In [8]:
#Import dataset
import pickle as p
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as plimg
from PIL import Image
def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb')as f:
        datadict = p.load(f,encoding='latin1')
        #print(datadict.keys())
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32)
        Y = np.array(Y)
        return X, Y

X1,Y1=load_CIFAR_batch('data_batch_1')
X2,Y2=load_CIFAR_batch('data_batch_2')
X3,Y3=load_CIFAR_batch('data_batch_3')
X4,Y4=load_CIFAR_batch('data_batch_4')
X5,Y5=load_CIFAR_batch('data_batch_5')
X_test,Y_test=load_CIFAR_batch('test_batch')
X=np.vstack((X1,X2,X3,X4,X5)).reshape(50000,3072)
X=X/256
Y=np.hstack((Y1,Y2,Y3,Y4,Y5))
x_test=X_test/256
x_test=x_test.reshape(10000,3072)

# cross validation
Iv = [[]] * 5
Ir = [[]] * 5
x_train,y_train,x_hold,y_hold=[[]]*5,[[]]*5,[[]]*5,[[]]*5
for m in range(5):
    Iv[m] = range(m,50000,5)
    Ir[m] = np.setdiff1d(range(50000),Iv[m])
    x_train[m]=X[Ir[m]]
    y_train[m]=Y[Ir[m]]
    y_train[m]=y_train[m].reshape(40000,1)
    x_hold[m]=X[Iv[m]]
    y_hold[m]=Y[Iv[m]]
    y_hold[m]=y_hold[m].reshape(10000,1)

# 1 vs all
w=[[]]*5
error_cross=[]
for n in range(5):
    w[n]=[[]]*10
    for k in range(10):
        y=y_train[n].copy()
        y[y_train[n]==k],y[y_train[n]!=k]=-1,1
        w[n][k]=np.linalg.inv(x_train[n].T@x_train[n])@x_train[n].T@y
    
    error=0
    for j in range(5000):
        result=[]
        for i in range(10):
            y=y_hold[n].copy()
            y[y_hold[n]==i],y[y_hold[n]!=i]=-1,1
            predict=x_hold[n][j]@w[n][i]
            result.append(predict)
            if predict<0:
                label=i
                break
            label=result.index(min(result))
        if label!=y_hold[n][j]:
            error+=1
    error_cross.append(error) 
        
w_best=w[error_cross.index(min(error_cross))]        

error=0
for m in range(10000):
    result=[]
    for n in range(10):
        predict=x_test[m]@w_best[n]
        result.append(predict)
        if predict<0:
            label=n
            break
        label=result.index(min(result))
    if label!=Y_test[m]:
        error+=1
print('error rate=\n',error/10000)
        
    

error rate=
 0.6493


AttributeError: 'list' object has no attribute 'shape'

15.872076103552715
6.371007204682382
8.444933699968797
10.272398880753185
1.820318804796905
6.636515455826949
8.30582928479692
11.733660187273605
9.404256192106027
10.306733934976556


In [27]:
d=np.array([[1],[2],[3],[1],[0],[1],[2]])
for i in range(2):
    c=d.copy()
    c[d==i],c[d!=i]=-1,1
    print('c=',c)
    print('d=',d)

c= [[ 1]
 [ 1]
 [ 1]
 [ 1]
 [-1]
 [ 1]
 [ 1]]
d= [[1]
 [2]
 [3]
 [1]
 [0]
 [1]
 [2]]
c= [[-1]
 [ 1]
 [ 1]
 [-1]
 [ 1]
 [-1]
 [ 1]]
d= [[1]
 [2]
 [3]
 [1]
 [0]
 [1]
 [2]]


In [35]:

# cross validation
Iv = [[]] * 5
Ir = [[]] * 5
x_train,y_train,x_hold,y_hold=[[]]*5,[[]]*5,[[]]*5,[[]]*5
for m in range(5):
    Iv[m] = range(m,50000,5)
    Ir[m] = np.setdiff1d(range(50000),Iv[m])
    x_train[m]=X[Ir[m]]
    y_train[m]=Y[Ir[m]]*10
    y_train[m]=y_train[m].reshape(40000,1)
    x_hold[m]=X[Iv[m]]
    y_hold[m]=Y[Iv[m]]*10
    y_hold[m]=y_hold[m].reshape(10000,1)
    x_train[m]=np.hstack((np.ones((x_train[m].shape[0],1)),x_train[m]))
    x_hold[m]=np.hstack((np.ones((x_hold[m].shape[0],1)),x_hold[m]))
    
#train
la_vals=np.linspace(10**(-5),100,20)
W_best=[[]]*5
for i in range(5):
    W=weight(x_train[i],y_train[i],la_vals)
    c=100
    for j in range(20):
        prediction=label(x_hold[i]@W[:,j:j+1]/10)
        error=np.mean(prediction!=y_hold[i])
        if error<c:
            c=error
            W_best[i]=W[:,j:j+1]
            
           
print('done')
# test
error_rate=[[]]*5
x_test=np.hstack((np.ones((x_test.shape[0],1)),x_test))
for j in range(5):
    result=label(x_test@W_best[j]/10)
    error_rate[j]=np.mean(result!=Y_test)
print(min(error_rate))
    

0.9


In [31]:
#Import dataset
import pickle as p
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as plimg
from PIL import Image
def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb')as f:
        datadict = p.load(f,encoding='latin1')
        #print(datadict.keys())
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32)
        Y = np.array(Y)
        return X, Y

def weight(A,y,la):
    p,n= A.shape
    num_lam = len(la)
    X = np.zeros((n, num_lam))
    for k,each_lamda in enumerate(la):
        X[:,k:k+1]=np.linalg.inv((A@A.T+each_lamda*np.eye(p)))@A.T@y
    return X
X1,Y1=load_CIFAR_batch('data_batch_1')
X2,Y2=load_CIFAR_batch('data_batch_2')
X3,Y3=load_CIFAR_batch('data_batch_3')
X4,Y4=load_CIFAR_batch('data_batch_4')
X5,Y5=load_CIFAR_batch('data_batch_5')
X_test,Y_test=load_CIFAR_batch('test_batch')
X=np.vstack((X1,X2,X3,X4,X5)).reshape(50000,3072)
Y=np.hstack((Y1,Y2,Y3,Y4,Y5))
x_test=X_test/25.6
x_test=x_test.reshape(10000,3072)

# cross validation
Iv = [[]] * 5
Ir = [[]] * 5
x_train,y_train,x_hold,y_hold=[[]]*5,[[]]*5,[[]]*5,[[]]*5
la_vals=np.linspace(10**(-5),100,20)
W_best=[[]]*5
for i in range(5):
    W=weight(x_train[i],y_train[i],la_vals)
    c=1
    for j in range(20):
        prediction=label(x_hold[i]@W[:,j:j+1])
        error=np.mean(prediction!=y_hold[i])
        if error<c:
            c=error
            W_best[i]=W[:,j]
            
for m in range(5):
    Iv[m] = range(m,50000,5)
    Ir[m] = np.setdiff1d(range(50000),Iv[m])
    x_train[m]=X[Ir[m]]/25.6  
    y_train[m]=Y[Ir[m]]
    x_hold[m]=X[Iv[m]]/25.6
    y_hold[m]=Y[Iv[m]]
    x_train[m]=np.hstack((np.ones((x_train[m].shape[0],1)),x_train[m]))
print(x_train[m].shape)
print(y_train[0].shape)


(40000, 3073)
(40000,)


In [22]:
def label(A):
    A=np.around(A,decimals=1)
    for k in range(A.shape[0]):
        if A[k]>9:
            A[k]=9
        if A[k]<0:
            A[k]=0
        if A[k]>0 and A[k]<9:
            A[k]=np.around(A[k],decimals=1)
B=np.array([[2.45],[7.21],[6.32],[8.97]])
print(label(B))

None


In [ ]:
error_rate=0
for j in range(5):
    result=label(x_test@W_best)
    error_rate=np.mean(result!=Y_test)
print(error_rate)
# Train
def label(A):
    for k in A:
        if k>9:
            k=9
        elif k<0:
            k=0
        else:
            k=np.round(k)
            
la_vals=np.linspace(10**(-5),100,20)
W_best=[[]]*5
for i in range(5):
    W=weight(x_train[i],y_train[i],la_vals)
    c=1
    for j in range(50):
        prediction=label(x_hold[i]@W[:,j])
        error=np.mean(prediction!=y_hold[i])
        if error<c:
            c=error
            W_best[i]=W[:,j]
            
print('done')
# test
error_rate=[[]]*5
for j in range(5):
    result=label(x_test@W_best[j])
    error_rate[j]=np.mean(result!=y_test)
print(min(error_rate))
        

In [ ]:
def weight(A,y,la):
    p,n= A.shape
    num_lam = len(la)
    X = np.zeros((n, num_lam))
    for k,each_lamda in enumerate(la):
        X[:,k:k+1]=np.linalg.inv((A@A.T+each_lamda*np.eye(p)))@A.T@y
    return X